In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkcalendar import Calendar
import json
import re
from tkinter import ttk
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Funktion zum Laden der Kundendaten aus der Datei
def lade_kunden(dateipfad="kunden.json"):
    try:
        with open(dateipfad, "r") as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return {}

# Funktion zur Validierung einer E-Mail-Adresse
def validate_email(email):
    regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(regex, email) is not None

# Funktion zur Validierung einer Telefonnummer
def validate_telefonnummer(telefonnummer):
    regex = r'^\+?\d{10,15}$'  # Beispiel für eine gültige Telefonnummer (international)
    return re.match(regex, telefonnummer) is not None

# Funktion zum Hinzufügen eines neuen Kunden
def kunden_zu_datenbank_hinzufuegen(kunden_daten, vorname, nachname, geburtsdatum, email, telefonnummer, dateipfad="kunden.json"):
    kunden_daten[vorname + " " + nachname] = {
        "vorname": vorname,
        "nachname": nachname,
        "geburtsdatum": geburtsdatum,
        "email": email,
        "telefonnummer": telefonnummer,
        "termine": []  # Initialisiert eine leere Liste für Termine
    }
    with open(dateipfad, "w") as f:
        json.dump(kunden_daten, f, ensure_ascii=False, indent=4)

# Funktion zur Suche eines Kunden
def kunde_suchen(kunden_daten, vorname, nachname, geburtsdatum):
    for full_name, details in kunden_daten.items():
        if details["vorname"].lower() == vorname.lower() and \
           details["nachname"].lower() == nachname.lower() and \
           details["geburtsdatum"] == geburtsdatum:
            return details
    return None

# Funktion zum Senden einer Bestätigungs-E-Mail (Simuliert)
def sende_termin_bestätigung(kunde_email, datum, uhrzeit):
    # Simuliert das Senden einer E-Mail
    print(f"E-Mail gesendet an {kunde_email} mit dem Termin: {datum} um {uhrzeit}")
    # In einer echten Anwendung könnte hier smtplib verwendet werden, um eine E-Mail zu versenden
    # Hier nur als Simulation:
    # 
    # msg = MIMEMultipart()
    # msg['From'] = 'your_email@example.com'
    # msg['To'] = kunde_email
    # msg['Subject'] = 'Terminbestätigung'
    # body = f"Ihr Termin wurde für den {datum} um {uhrzeit} bestätigt."
    # msg.attach(MIMEText(body, 'plain'))
    #
    # server = smtplib.SMTP('smtp.example.com', 587)
    # server.starttls()
    # server.login('your_email@example.com', 'your_password')
    # text = msg.as_string()
    # server.sendmail('your_email@example.com', kunde_email, text)
    # server.quit()

# GUI Klasse
class KundenApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Kundenverwaltung mit Terminkalender")
        self.kunden_daten = lade_kunden()

        # Suchformular
        self.suchen_frame = tk.LabelFrame(self.root, text="Kunde suchen", padx=10, pady=10)
        self.suchen_frame.grid(row=0, column=0, padx=10, pady=10)

        tk.Label(self.suchen_frame, text="Vorname:").grid(row=0, column=0)
        self.suchen_vorname = tk.Entry(self.suchen_frame)
        self.suchen_vorname.grid(row=0, column=1)

        tk.Label(self.suchen_frame, text="Nachname:").grid(row=1, column=0)
        self.suchen_nachname = tk.Entry(self.suchen_frame)
        self.suchen_nachname.grid(row=1, column=1)

        tk.Label(self.suchen_frame, text="Geburtsdatum (TT.MM.JJJJ):").grid(row=2, column=0)
        self.suchen_geburtsdatum = tk.Entry(self.suchen_frame)
        self.suchen_geburtsdatum.grid(row=2, column=1)

        self.suchen_button = tk.Button(self.suchen_frame, text="Suchen", command=self.suchen)
        self.suchen_button.grid(row=3, column=0, columnspan=2, pady=10)

        # Hinzufügenformular
        self.hinzufuegen_frame = tk.LabelFrame(self.root, text="Neuen Kunden hinzufügen", padx=10, pady=10)
        self.hinzufuegen_frame.grid(row=1, column=0, padx=10, pady=10)

        tk.Label(self.hinzufuegen_frame, text="Vorname:").grid(row=0, column=0)
        self.hinzufuegen_vorname = tk.Entry(self.hinzufuegen_frame)
        self.hinzufuegen_vorname.grid(row=0, column=1)

        tk.Label(self.hinzufuegen_frame, text="Nachname:").grid(row=1, column=0)
        self.hinzufuegen_nachname = tk.Entry(self.hinzufuegen_frame)
        self.hinzufuegen_nachname.grid(row=1, column=1)

        tk.Label(self.hinzufuegen_frame, text="Geburtsdatum (TT.MM.JJJJ):").grid(row=2, column=0)
        self.hinzufuegen_geburtsdatum = tk.Entry(self.hinzufuegen_frame)
        self.hinzufuegen_geburtsdatum.grid(row=2, column=1)

        tk.Label(self.hinzufuegen_frame, text="E-Mail:").grid(row=3, column=0)
        self.hinzufuegen_email = tk.Entry(self.hinzufuegen_frame)
        self.hinzufuegen_email.grid(row=3, column=1)

        tk.Label(self.hinzufuegen_frame, text="Telefonnummer:").grid(row=4, column=0)
        self.hinzufuegen_telefonnummer = tk.Entry(self.hinzufuegen_frame)
        self.hinzufuegen_telefonnummer.grid(row=4, column=1)

        self.hinzufuegen_button = tk.Button(self.hinzufuegen_frame, text="Hinzufügen", command=self.hinzufuegen)
        self.hinzufuegen_button.grid(row=5, column=0, columnspan=2, pady=10)

        # Terminbuchungsformular
        self.termin_frame = tk.LabelFrame(self.root, text="Terminbuchung", padx=10, pady=10)
        self.termin_frame.grid(row=2, column=0, padx=10, pady=10)

        tk.Label(self.termin_frame, text="Wählen Sie einen Kunden aus:").grid(row=0, column=0)

        self.kunden_dropdown = ttk.Combobox(self.termin_frame)
        self.kunden_dropdown.grid(row=0, column=1)

        self.update_kunden_dropdown()

        tk.Label(self.termin_frame, text="Wählen Sie das Datum:").grid(row=1, column=0)
        self.calendar = Calendar(self.termin_frame, selectmode='day', date_pattern='dd/mm/yyyy')
        self.calendar.grid(row=1, column=1)

        tk.Label(self.termin_frame, text="Wählen Sie die Uhrzeit:").grid(row=2, column=0)
        self.stunden_dropdown = ttk.Combobox(self.termin_frame, values=[str(i).zfill(2) for i in range(24)])
        self.stunden_dropdown.grid(row=2, column=1)

        self.minuten_dropdown = ttk.Combobox(self.termin_frame, values=[str(i).zfill(2) for i in range(0, 60, 5)])
        self.minuten_dropdown.grid(row=2, column=2)

        self.termin_button = tk.Button(self.termin_frame, text="Termin buchen", command=self.termin_buchen)
        self.termin_button.grid(row=3, column=0, columnspan=3, pady=10)

    def update_kunden_dropdown(self):
        kunden_namen = [f"{details['vorname']} {details['nachname']}" for details in self.kunden_daten.values()]
        self.kunden_dropdown['values'] = kunden_namen

    def suchen(self):
        vorname = self.suchen_vorname.get()
        nachname = self.suchen_nachname.get()
        geburtsdatum = self.suchen_geburtsdatum.get()

        kunde = kunde_suchen(self.kunden_daten, vorname, nachname, geburtsdatum)
        if kunde:
            messagebox.showinfo("Kunde gefunden", f"Vorname: {kunde['vorname']}\nNachname: {kunde['nachname']}\nGeburtsdatum: {kunde['geburtsdatum']}\nE-Mail: {kunde['email']}\nTelefonnummer: {kunde['telefonnummer']}")
        else:
            messagebox.showerror("Kunde nicht gefunden", "Der angegebene Kunde konnte nicht gefunden werden.")

    def hinzufuegen(self):
        vorname = self.hinzufuegen_vorname.get()
        nachname = self.hinzufuegen_nachname.get()
        geburtsdatum = self.hinzufuegen_geburtsdatum.get()
        email = self.hinzufuegen_email.get()
        telefonnummer = self.hinzufuegen_telefonnummer.get()

        if not validate_email(email):
            messagebox.showerror("Fehler", "Ungültige E-Mail-Adresse.")
            return
        if not validate_telefonnummer(telefonnummer):
            messagebox.showerror("Fehler", "Ungültige Telefonnummer.")
            return

        kunden_zu_datenbank_hinzufuegen(self.kunden_daten, vorname, nachname, geburtsdatum, email, telefonnummer)
        self.update_kunden_dropdown()
        messagebox.showinfo("Kunde hinzugefügt", f"Kunde {vorname} {nachname} wurde erfolgreich hinzugefügt.")
        
    
    def termin_buchen(self):
        ausgewählter_kunde = self.kunden_dropdown.get()
        if not ausgewählter_kunde:
            messagebox.showerror("Fehler", "Bitte wählen Sie einen Kunden aus.")
            return

        datum = self.calendar.get_date()
        stunde = self.stunden_dropdown.get()
        minute = self.minuten_dropdown.get()

        if not stunde or not minute:
            messagebox.showerror("Fehler", "Bitte wählen Sie eine Uhrzeit aus.")
            return

        # Termin speichern
        for name, details in self.kunden_daten.items():
            if f"{details['vorname']} {details['nachname']}" == ausgewählter_kunde:
                termin = {"datum": datum, "uhrzeit": f"{stunde}:{minute}"}
                details["termine"].append(termin)
                with open("kunden.json", "w") as f:
                    json.dump(self.kunden_daten, f, ensure_ascii=False, indent=4)
                
                # Bestätigungs-E-Mail an den Kunden senden
                kunde_email = details["email"]
                sende_termin_bestätigung(kunde_email, datum, f"{stunde}:{minute}")
                
                messagebox.showinfo("Termin gebucht", f"Termin für {ausgewählter_kunde} am {datum} um {stunde}:{minute} wurde erfolgreich gebucht.\nE-Mail zur Bestätigung wurde gesendet.")
                return

# Hauptfenster erstellen
root = tk.Tk()
app = KundenApp(root)
root.mainloop()